<a href="https://colab.research.google.com/github/DhawalV1/qhack2024/blob/main/Electronic_structure_ground_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit==0.44.0

In [ ]:
pip install qiskit-nature

In [ ]:
pip install --prefer-binary pyscf

In [4]:
from qiskit import *
## Obtaining the hamiltonian (energy operator)
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver, Psi4Driver
# NOTE From hwa: PySCF runs only on all Linux, and some MacOS; No support for Windows

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [13]:
problem = driver.run()
print(problem)
problem.__dict__

{'_hamiltonian': <qiskit_nature.second_q.hamiltonians.electronic_energy.ElectronicEnergy at 0x78dfb3c584c0>,
 'properties': <qiskit_nature.second_q.problems.electronic_properties_container.ElectronicPropertiesContainer at 0x78dfb3c5a1a0>,
 'molecule': MoleculeInfo(symbols=['H', 'H'], coords=[(0.0, 0.0, 0.0), (0.0, 0.0, 1.3889487015553204)], multiplicity=1, charge=0, units=<DistanceUnit.BOHR: 'Bohr'>, masses=[1, 1]),
 'basis': <ElectronicBasis.MO: 'molecular'>,
 '_num_particles': (1, 1),
 'num_spatial_orbitals': 2,
 '_orbital_occupations': None,
 '_orbital_occupations_b': None,
 'reference_energy': -1.116998996754004,
 'orbital_energies': array([-0.58062892,  0.67633625]),
 'orbital_energies_b': None}

In [15]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()
mapper.__dict__

{}

In [16]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
numpy_solver.__dict__

{'_eigensolver': <qiskit_algorithms.eigensolvers.numpy_eigensolver.NumPyEigensolver at 0x78dfb3c5be50>}

In [8]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters


In [17]:
vqe_solver.__dict__

{'estimator': <qiskit.primitives.estimator.Estimator at 0x78dfc0227610>,
 'ansatz': <qiskit_nature.second_q.circuit.library.ansatzes.uccsd.UCCSD at 0x78dfc0227c70>,
 'optimizer': <qiskit_algorithms.optimizers.slsqp.SLSQP at 0x78dfc0226fe0>,
 'gradient': None,
 '_initial_point': [0.0, 0.0, 0.0],
 'callback': None}

In [9]:
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(Estimator(), tl_circuit, SLSQP())

In [18]:
another_solver.__dict__

{'estimator': <qiskit.primitives.estimator.Estimator at 0x78dffcdb60e0>,
 'ansatz': <qiskit.circuit.library.n_local.two_local.TwoLocal at 0x78dfc0227880>,
 'optimizer': <qiskit_algorithms.optimizers.slsqp.SLSQP at 0x78dfc0226ce0>,
 'gradient': None,
 '_initial_point': None,
 'callback': None}

In [10]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

In [25]:
res = calc.solve(problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030144
  - computed part:      -1.857275030144
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035695
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948929727]
    - computed part:      [0.0  0.0  1.388948929727]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000229727]  Total: 0.000000229727
                 (debye): [0.0  0.0  -0.000000583907]  Total: 0.000000583907
 


In [27]:
calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


In [28]:
hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)

Polynomial Tensor
 "+-":
array([[-1.25633907e+00, -1.37083854e-17],
       [-6.07732712e-17, -4.71896007e-01]])
 "++--":
array([6.75710155e-01, 1.39486891e-16, 1.80931200e-01, 6.64581730e-01,
       2.57172666e-16, 6.98573723e-01])


In [14]:
calc.__dict__

{'_qubit_mapper': <qiskit_nature.second_q.mappers.jordan_wigner_mapper.JordanWignerMapper at 0x78dffcdb43d0>,
 '_solver': <qiskit_algorithms.minimum_eigensolvers.vqe.VQE at 0x78dfc0226bc0>}

In [19]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [20]:
result = solver.solve(problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


UsageError: Line magic function `%version_table` not found.
